In [127]:
import xarray as xr
import numpy as np
import pandas as pd
import os
from io import StringIO
from collections import OrderedDict

In [128]:
def strip_line(s):
    s = s.strip()
    if s.startswith("'"):
        istart = 1
        iend = s.find("'", 1)
    else:
        istart = 0
        iend = s.find(' ')
    return s[istart:iend]

def read_file_manager(fman):
    with open(fman, "r") as f:
        lines = f.readlines()
        lines = [strip_line(l) for l in lines if l[0] != '!']
        i = 0
        keys = ['fman_ver', 'setting_path', 'input_path', 'output_path', 'decision', 
                'meta_time', 'meta_attr', 'meta_type', 'meta_force', 'meta_localpar',
                'ouput_file', 'meta_index', 'meta_basinpar', 'meta_basinvar',
                'local_attr', 'local_par', 'basin_par', 'forcing_list', 'initial_cond',
                'para_trial', 'output_prefix']
        file_man = OrderedDict(zip(keys, lines))
        
        return file_man

    
    
%time fl = read_file_manager('settings_org/syntheticTestCases/celia1990/summa_fileManager_celia1990.txt')
fl   

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 868 µs


OrderedDict([('fman_ver', 'SUMMA_FILE_MANAGER_V1.'),
             ('setting_path', '<BASEDIR>/settings/'),
             ('input_path',
              '<BASEDIR>/testCases_data/inputData/syntheticData/celia1990/'),
             ('output_path', '<BASEDIR>/output/syntheticTestCases/celia1990/'),
             ('decision',
              'syntheticTestCases/celia1990/summa_zDecisions_celia1990.txt'),
             ('meta_time', 'meta/summa_zTimeMeta.txt'),
             ('meta_attr', 'meta/summa_zLocalAttributeMeta.txt'),
             ('meta_type', 'meta/summa_zCategoryMeta.txt'),
             ('meta_force', 'meta/summa_zForceMeta.txt'),
             ('meta_localpar', 'meta/summa_zLocalParamMeta.txt'),
             ('ouput_file', 'meta/summa_zLocalModelVarMeta.txt'),
             ('meta_index', 'meta/summa_zLocalModelIndexMeta.txt'),
             ('meta_basinpar', 'meta/summa_zBasinParamMeta.txt'),
             ('meta_basinvar', 'meta/summa_zBasinModelVarMeta.txt'),
             ('local_attr',


TypeError: 'builtin_function_or_method' object is not subscriptable

In [83]:
# find all inital state files
def search_ini_files():
    os.chdir('/home/mgou/Dropbox/postdoc/summa/summaTestCases')
    ini_files = []
    for dirpath, dirnames, filenames in os.walk('settings_org'):
        for f in filenames:
            if 'summa_zInitialCond' in f:
                ini_files.append(os.path.join(dirpath,f))
                os.listdir(dirpath)
%timeit search_ini_files()

1000 loops, best of 3: 1.56 ms per loop


In [82]:
# find all inital state files
def search_ini_files():
    os.chdir('/home/mgou/Dropbox/postdoc/summa/summaTestCases')
    ini_files = []
    for dirpath, dirnames, filenames in os.walk('settings_org'):
        for f in filenames:
            ini_files.append(os.path.join(dirpath,f)) if 'summa_zInitialCond' in str(f) else 'pass'
            
%timeit search_ini_files()

1000 loops, best of 3: 1.54 ms per loop


In [84]:
ini_files

['settings_org/syntheticTestCases/celia1990/summa_zInitialCond_celia1990.txt',
 'settings_org/syntheticTestCases/miller1998/summa_zInitialCond_miller1998.txt',
 'settings_org/syntheticTestCases/miller1998/summa_zInitialCond_millerClay.txt',
 'settings_org/syntheticTestCases/miller1998/summa_zInitialCond_millerLoam.txt',
 'settings_org/syntheticTestCases/miller1998/summa_zInitialCond_millerSand.txt',
 'settings_org/syntheticTestCases/mizoguchi1990/summa_zInitialCond_mizoguchi.txt',
 'settings_org/syntheticTestCases/wigmosta1999/summa_zInitialCond-orig.txt',
 'settings_org/syntheticTestCases/wigmosta1999/summa_zInitialCond.txt',
 'settings_org/wrrPaperTestCases/figure01/summa_zInitialCond.txt',
 'settings_org/wrrPaperTestCases/figure02/summa_zInitialCond.txt',
 'settings_org/wrrPaperTestCases/figure03/summa_zInitialCond.txt',
 'settings_org/wrrPaperTestCases/figure05/summa_zInitialCond.txt',
 'settings_org/wrrPaperTestCases/figure06/summa_zInitialCond.txt',
 'settings_org/wrrPaperTestCas

In [92]:
ini_file = initial_files[0]
def read_ini_file(ini_file):
    scalar_var_line = ''
    layer_var_line = ''
    scalar_var_block = False
    layer_var_block = False
    with open(ini_file, "r") as f:
        lines = f.readlines()
        for l in lines:
            if l.startswith('!'):
                continue
            if 'start_scalar_icond' in l:
                scalar_var_block = True
                continue
            if 'end_scalar_icond' in l:
                scalar_var_block = False
                continue
            if 'start_layer_icond' in l:
                layer_var_block = True
                continue
            if 'end_layer_icond' in l:
                layer_var_block = False
                break
            if scalar_var_block:
                scalar_var_line += l
            if layer_var_block:
                layer_var_line += l
                
    scalar_var = pd.read_table(StringIO(scalar_var_line), sep=' ', skipinitialspace=True, header=None)
    scalar_var.columns=['var', 'val']    
    layer_var = pd.read_table(StringIO(layer_var_line), sep=' ', skipinitialspace=True)
    return (scalar_var, layer_var) 

scalar_var, layer_var = read_ini_file(ini_file)

In [93]:
scalar_var

,var,val
0,dt_init,10.00
1,scalarCanopyIce,0.00
2,scalarCanopyLiq,0.00
3,scalarCanairTemp,286.00
4,scalarCanopyTemp,290.00
5,scalarSnowAlbedo,0.82
6,scalarSWE,0.00
7,scalarSnowDepth,0.00
8,scalarSfcMeltPond,0.00
9,scalarAquiferStorage,0.00


In [94]:
layer_var

,layerType,iLayerHeight,mLayerDepth,mLayerTemp,mLayerVolFracIce,mLayerVolFracLiq,mLayerMatricHead
0,soil,0.000,0.006,285.16,0.0,0.116750,-5.375
1,soil,0.006,0.006,285.16,0.0,0.109937,-10.000
2,soil,0.012,0.006,285.16,0.0,0.109937,-10.000
3,soil,0.018,0.006,285.16,0.0,0.109937,-10.000
4,soil,0.024,0.006,285.16,0.0,0.109937,-10.000
5,soil,0.030,0.006,285.16,0.0,0.109937,-10.000
6,soil,0.036,0.006,285.16,0.0,0.109937,-10.000
7,soil,0.042,0.006,285.16,0.0,0.109937,-10.000
8,soil,0.048,0.006,285.16,0.0,0.109937,-10.000
9,soil,0.054,0.006,285.16,0.0,0.109937,-10.000


In [2]:
nhru = 11723
nspec = 2
nmidSoil = 8
nifcSoil = nmidSoil + 1
nmidToto = 8
nifcToto = nmidToto + 1
nscalarv = 1

nSnow                = np.empty([nscalarv, nhru], dtype = np.int16)
nSoil                = np.empty([nscalarv, nhru], dtype = np.int16)
dt_init              = np.empty([nscalarv, nhru])
scalarCanopyIce      = np.empty([nscalarv, nhru])
scalarCanopyLiq      = np.empty([nscalarv, nhru])
scalarCanairTemp     = np.empty([nscalarv, nhru])
scalarCanopyTemp     = np.empty([nscalarv, nhru])
scalarSnowAlbedo     = np.empty([nscalarv, nhru])
scalarSWE            = np.empty([nscalarv, nhru])
scalarSnowDepth      = np.empty([nscalarv, nhru])
scalarSfcMeltPond    = np.empty([nscalarv, nhru])
scalarAquiferStorage = np.empty([nscalarv, nhru])
iLayerHeight         = np.empty([nifcToto, nhru])
mLayerDepth          = np.empty([nmidToto, nhru])
mLayerTemp           = np.empty([nmidToto, nhru])
mLayerVolFracIce     = np.empty([nmidToto, nhru])
mLayerVolFracLiq     = np.empty([nmidToto, nhru])
mLayerMatricHead     = np.empty([nmidSoil, nhru])

In [22]:
nSnow.fill(0)
nSoil.fill(8)
dt_init.fill(360.0)
scalarCanopyIce.fill(0.0)
scalarCanopyLiq.fill(1.0)
scalarCanairTemp.fill(275.0)
scalarCanopyTemp.fill(275.0)
scalarSnowAlbedo.fill(0.82)
scalarSWE.fill(0.0)
scalarSnowDepth.fill(0.0)
scalarSfcMeltPond.fill(0.0)
scalarAquiferStorage.fill(0.0)

In [23]:
iLayerHeight = np.repeat([0, 0.025, 0.1, 0.25, 0.5, 1.0, 1.5, 2.5, 4.0], nhru).reshape([nifcToto, nhru])
mLayerDepth  = np.repeat([0.025, 0.075, 0.15, 0.25, 0.5, 0.5, 1.0, 1.5], nhru).reshape([nmidToto, nhru])
mLayerTemp.fill(283.5)
mLayerVolFracIce.fill(0.0)
mLayerVolFracLiq.fill(0.3)
mLayerMatricHead.fill(-10.0)

In [24]:
ini_nc = xr.Dataset({'nSnow':(['scalarv', 'hru'], nSnow),
                     'nSoil':(['scalarv', 'hru'], nSoil),
                     'dt_init':(['scalarv', 'hru'], dt_init),
                     'scalarCanopyIce':(['scalarv', 'hru'], scalarCanopyIce),
                     'scalarCanopyLiq':(['scalarv', 'hru'], scalarCanopyLiq),
                     'scalarCanairTemp':(['scalarv', 'hru'], scalarCanairTemp),
                     'scalarCanopyTemp':(['scalarv', 'hru'], scalarCanopyTemp),
                     'scalarSnowAlbedo':(['scalarv', 'hru'], scalarSnowAlbedo),
                     'scalarSWE':(['scalarv', 'hru'], scalarSWE),
                     'scalarSnowDepth':(['scalarv', 'hru'], scalarSnowDepth),
                     'scalarSfcMeltPond':(['scalarv', 'hru'], scalarSfcMeltPond),
                     'scalarAquiferStorage':(['scalarv', 'hru'], scalarAquiferStorage),
                     'iLayerHeight':(['ifcToto','hru'], iLayerHeight),
                     'mLayerDepth':(['midToto', 'hru'], mLayerDepth),
                     'mLayerTemp':(['midToto', 'hru'], mLayerTemp),
                     'mLayerVolFracIce':(['midToto', 'hru'], mLayerVolFracIce),
                     'mLayerVolFracLiq':(['midToto', 'hru'], mLayerVolFracLiq),
                     'mLayerMatricHead':(['midSoil', 'hru'], mLayerMatricHead)})

In [25]:
ini_nc.to_netcdf(r'D:\Cloud\Dropbox\postdoc\summa\summaData\summa_run\ini_crb.nc', format = 'NETCDF4_CLASSIC')

In [18]:
12*(2011-1915+1) / 11 

105.81818181818181

In [21]:
11723/16/11

66.60795454545455